In [98]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [99]:
def getDist(point1, point2) :
    sum = 0
    for i in range(828) : 
        sum += (point1[i] - point2[i])**2
    return (math.sqrt(sum))

In [100]:
def getImgTrainData() :
    train_all = []
    for class_type in ["coast","forest","highway","mountain","opencountry"] : 
        X = [np.array([]).reshape(0,23)] * 36
        string = f"features\{class_type}" + r"\train"
        for image in os.listdir(string) : 
            file = open(string+f"\{image}","r")
            ind = 0
            for line in file: 
                temp_list = line.split(" ")
                temp_array = []
                for temp in temp_list:
                    temp_array.append(temp.split("\n")[0])
                temp_array = np.array(temp_array, dtype="float")
                X[ind] = np.r_["0,2", X[ind], temp_array]
                ind += 1
        train_all.append(X)  
    return train_all

In [101]:
def getImgTestData() :
    test_all = []
    for class_type in ["coast","forest","highway","mountain","opencountry"] : 
        X = []
        string = f"features\{class_type}" + r"\dev"
        for image in os.listdir(string) : 
            Y = []
            file = open(string+f"\{image}","r")
            for line in file: 
                temp_list = line.split(" ")
                temp_array = []
                for temp in temp_list:
                    temp_array.append(temp.split("\n")[0])
                temp_array = np.array(temp_array, dtype="float")
                Y.append(temp_array)
            X.append(Y)
        test_all.append(X)  
    return test_all

In [102]:
def getSynTrainData() :
    data = np.genfromtxt(r"synthetic\train.txt", delimiter=",")
    return data

In [103]:
def getSynDevData() :
    data = np.genfromtxt(r"synthetic\dev.txt", delimiter=",")
    return data

In [123]:
def KMeansTrain(K, X) :
    num_samples = X.shape[0]
    num_features = X.shape[1]

    centroids = np.array([]).reshape(num_features, 0)
    for k in range(K):
        centroids = np.c_[centroids, X[random.randint(0, num_samples-1)]]

    for i in range(10):
        distances = np.array([]).reshape(num_samples, 0)
        for k in range(K):
            distances = np.c_[distances, np.sum((X - centroids[:,k])**2, axis=1)]

        cluster_numbers = np.argmin(distances, axis=1)
        
        clusters = [np.array([]).reshape(0, num_features)] * K

        for n in range(num_samples):
            clusters[cluster_numbers[n]] = np.r_["0,2", clusters[cluster_numbers[n]], X[n]]

        for k in range(K):
            centroids[:,k] = np.nanmean(clusters[k], axis=0)
    
    return centroids

In [105]:
def KMeansTest(X, centroids) : 
    num_samples = X.shape[0]
    K = centroids.shape[1]
    distances = np.array([]).reshape(num_samples, 0)
    for k in range(K) : 
        distances = np.c_[distances, np.sum((X - centroids[:,k])**2, axis=1)]
    
    cluster_numbers = np.argmin(distances, axis=1)
    return cluster_numbers 

In [106]:
def KMeansTestImg(X, centroids) : 
    K = centroids.shape[1]
    distances = np.array([]).reshape(1, 0)
    for k in range(K) : 
        distances = np.c_[distances, np.sum((X - centroids[:,k])**2)]
    
    cluster_numbers = np.argmin(distances, axis=1)
    return cluster_numbers 

In [107]:
def synData() :
    syn_input = getSynTrainData()
    syn_data = np.array(syn_input[:,:2], dtype="float")
    syn_data1 = np.array(syn_data[:1250,:])
    syn_data2 = np.array(syn_data[1250:,:])
    syn_class = np.array(syn_input[:,2], dtype="int")

    K = 10
    syn_centroids1 = KMeansTrain(K, syn_data1)
    syn_centroids2 = KMeansTrain(K, syn_data2)
    syn_centroids = np.c_[syn_centroids1, syn_centroids2]

    syn_test_input = getSynDevData()
    syn_test1 = np.array(syn_test_input[:500,:2], dtype="float")
    syn_test2 = np.array(syn_test_input[500:,:2], dtype="float")

    true = 0
    cluster_numbers1 = KMeansTest(syn_test1, syn_centroids)
    cluster_numbers2 = KMeansTest(syn_test2, syn_centroids)

    for i in range(500) : 
        if(cluster_numbers1[i] < K) :
            true += 1
        if(cluster_numbers2[i] >= K) :
            true += 1

    print(true)

In [126]:
# def ImgData() :
img_input = getImgTrainData()
K = 10
centroids = [np.array([]).reshape(23,0)] * 36

for i in range(5) : 
    for j in range(36) : 
        centroids[j] = np.c_[centroids[j], KMeansTrain(K, img_input[i][j])]

# print(centroids)

# # print(centroids[0])
img_test = getImgTestData() 
true = 0
blah = [0] * 5
class_blah = [0] * 5
total_count = 0
for i in range(5) : 
    for test_image in img_test[i] : 
        count = [0] * 5
        for j in range(36) : 
            cluster_number = KMeansTestImg(test_image[j], centroids[j])[0]
            count[int(cluster_number/K)] += 1
        test_class = np.argmax(count)
        if(test_class == i) :
            true += 1
            blah[i] += 1
        class_blah[i] += 1
        total_count += 1
print(f"accuracy is {100 * true/total_count}")   
print(blah)
print(class_blah)
print(total_count)

C:\Users\aswin\AppData\Local\Temp/ipykernel_7932/54075892.py:22: RuntimeWarning: Mean of empty slice
  centroids[:,k] = np.nanmean(clusters[k], axis=0)


accuracy is 43.39080459770115
[56, 57, 8, 23, 7]
[73, 66, 52, 75, 82]
348


In [7]:
# if __name__ == "__main__" :
    # training_data = getImgTrainData()
    # print(getDist(training_data[0][0], training_data[0][1]))